In [2]:
#debut de la run

import pickle
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
from torchvision import transforms
import numpy as np

class PKLVideoDataset(Dataset):
    def __init__(self, x_files, y_files, extractor, target_frames=16):
        self.videos = []
        self.labels = []
        self.extractor = extractor
        self.target_frames = target_frames

        for xf, yf in zip(x_files, y_files):
            with open(xf, "rb") as f:
                self.videos.extend(pickle.load(f))
            with open(yf, "rb") as f:
                self.labels.extend(pickle.load(f))

    def __len__(self):
        return len(self.videos)

    def __getitem__(self, idx):
        video = self.videos[idx]
        label = self.labels[idx]

        # Convertir BGR -> RGB
        frames = [cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in video]

        # Ajuster à target_frames
        if len(frames) < self.target_frames:
            repeat_factor = int(np.ceil(self.target_frames / len(frames)))
            frames = (frames * repeat_factor)[:self.target_frames]
        else:
            frames = frames[:self.target_frames]

        # Préparation avec l’extractor (gère resize, normalization, etc.)
        inputs = self.extractor(frames, return_tensors="pt")
        pixel_values = inputs["pixel_values"].squeeze(0)  # (C, T, H, W)

        return pixel_values, torch.tensor(label, dtype=torch.long)


x_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train1_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train2_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train3_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train4_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train5_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train6_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_train7_small3.pkl",

]
y_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train1.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train2.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train4.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train5.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train6.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_train7.pkl",

]
# Fichiers de validation
x_val_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val1_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val2_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val3_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val4_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val5_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val6_small3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/X_val7_small3.pkl",

]
y_val_files = [
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val1.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val2.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val3.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val4.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val5.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val6.pkl",
    "/content/drive/MyDrive/Projet non-alternant/Vidéo/ensemble_non_hasard/y_val7.pkl",

]




from google.colab import drive

# ============================================================
# 2️⃣ MONTAGE DU DRIVE ET CHARGEMENT DU DATAFRAME
# ============================================================
drive.mount("/content/drive", force_remount=True)
from transformers import VideoMAEFeatureExtractor

extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")

train_dataset = PKLVideoDataset(x_files, y_files, extractor=extractor)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

val_dataset = PKLVideoDataset(x_val_files, y_val_files, extractor=extractor)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=2)


Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:30: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(


In [ ]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from tqdm import tqdm
from transformers import VideoMAEFeatureExtractor, VideoMAEForVideoClassification
from torch.cuda.amp import autocast, GradScaler

# ==========================================
# 1️⃣ Modèle et extracteur
# ==========================================
extractor = VideoMAEFeatureExtractor.from_pretrained("MCG-NJU/videomae-base")
model = VideoMAEForVideoClassification.from_pretrained(
    "MCG-NJU/videomae-base",
    num_labels=20
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ==========================================
# 2️⃣ Optimiseur, critère, scaler
# ==========================================
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()
scaler = GradScaler()

# ==========================================
# 3️⃣ Entraînement complet
# ==========================================
num_epochs = 20
best_acc = 0.0
model_path = "videomae_best_model.pth"

train_acc_list, val_acc_list, train_loss_list, val_loss_list = [], [], [], []

for epoch in range(num_epochs):
    model.train()
    running_train_loss, running_train_correct, total_train = 0.0, 0, 0

    for pixel_values, labels in tqdm(train_loader, desc=f"Époque {epoch+1}/{num_epochs}"):
        pixel_values = pixel_values.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        with autocast():
            outputs = model(pixel_values)
            loss = criterion(outputs.logits, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_train_loss += loss.item()
        preds = outputs.logits.argmax(1)
        running_train_correct += (preds == labels).sum().item()
        total_train += labels.size(0)

    train_loss = running_train_loss / len(train_loader)
    train_acc = running_train_correct / total_train

    # ================= Validation =================
    model.eval()
    running_val_loss, running_val_correct, total_val = 0.0, 0, 0
    with torch.no_grad():
        for pixel_values, labels in val_loader:
            pixel_values = pixel_values.to(device)
            labels = labels.to(device)
            with autocast():
                outputs = model(pixel_values)
                loss = criterion(outputs.logits, labels)
            running_val_loss += loss.item()
            preds = outputs.logits.argmax(1)
            running_val_correct += (preds == labels).sum().item()
            total_val += labels.size(0)

    val_loss = running_val_loss / len(val_loader)
    val_acc = running_val_correct / total_val

    train_acc_list.append(train_acc)
    val_acc_list.append(val_acc)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

    print(f"📊 [Époque {epoch+1}/{num_epochs}] "
          f"Train Acc: {train_acc:.4f} | Train Loss: {train_loss:.4f} | "
          f"Val Acc: {val_acc:.4f} | Val Loss: {val_loss:.4f}")

    # ================= Sauvegarde du meilleur modèle =================
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f"💾 Nouveau meilleur modèle sauvegardé avec Val Acc = {best_acc:.4f}")

print("✅ Entraînement terminé !")
